In [ ]:
class DecisionTreeClassifier():
    def __init__(self):
        pass

    def __loss(self):
        pass
    
    def __split_node(self):
        pass
        
    def fit(self, X:np.ndarray):
        self.X = X
        self.thresholds = []
        self.features = []
        
    def predict(self):
        pass